In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import Imputer

import copy
import argparse

%matplotlib inline

In [2]:
nba_master = pd.read_csv('./all_seasons.csv')

In [8]:
nba_master.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Chris Robinson,VAN,23.0,195.58,90.71840,Western Kentucky,USA,1996,2,...,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97
1,1,Matt Fish,MIA,27.0,210.82,106.59412,North Carolina-Wilmington,USA,1992,2,...,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,2,Matt Bullard,HOU,30.0,208.28,106.59412,Iowa,USA,Undrafted,Undrafted,...,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,3,Marty Conlon,BOS,29.0,210.82,111.13004,Providence,USA,Undrafted,Undrafted,...,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,4,Martin Muursepp,DAL,22.0,205.74,106.59412,None,USA,1996,1,...,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97


In [9]:
nba_master['ts_pts'] = nba_master['ts_pct'] * nba_master['pts']

In [11]:
nba_master.sort_values(by='ts_pts', ascending=False)

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,ts_pts
7702,7702,Kevin Durant,OKC,25.0,205.74,108.862080,Texas,USA,2007,1,...,7.4,5.5,8.0,0.022,0.184,0.327,0.635,0.259,2013-14,20.3200
8744,8744,Stephen Curry,GSW,28.0,190.50,86.182480,Davidson,USA,2009,1,...,5.4,6.7,18.3,0.029,0.131,0.320,0.669,0.315,2015-16,20.1369
4119,4119,Kobe Bryant,LAL,27.0,198.12,99.790240,None,USA,1996,1,...,5.3,4.5,4.7,0.026,0.127,0.384,0.559,0.228,2005-06,19.7886
4495,4495,Kobe Bryant,LAL,28.0,198.12,99.790240,None,USA,1996,1,...,5.7,5.4,0.6,0.028,0.132,0.333,0.580,0.249,2006-07,18.3280
6166,6166,Kevin Durant,OKC,21.0,205.74,104.326160,Texas,USA,2007,1,...,7.6,2.8,7.0,0.038,0.177,0.317,0.607,0.130,2009-10,18.2707
7362,7362,Kevin Durant,OKC,24.0,205.74,106.594120,Texas,USA,2007,1,...,7.9,4.6,12.7,0.018,0.201,0.298,0.647,0.213,2012-13,18.1807
2778,2778,Tracy McGrady,ORL,24.0,203.20,95.254320,None,USA,1997,1,...,6.5,5.5,3.5,0.046,0.147,0.350,0.564,0.282,2002-03,18.1044
9461,9461,Isaiah Thomas,BOS,28.0,175.26,83.914520,Washington,USA,2011,2,...,2.7,5.9,5.0,0.019,0.070,0.337,0.625,0.305,2016-17,18.0625
6129,6129,LeBron James,CLE,25.0,203.20,113.398000,None,USA,2003,1,...,7.3,8.6,10.8,0.030,0.185,0.333,0.604,0.398,2009-10,17.9388
4321,4321,Allen Iverson,PHI,31.0,182.88,74.842680,Georgetown,USA,1996,1,...,3.2,7.4,0.8,0.016,0.071,0.354,0.543,0.331,2005-06,17.9190
